# Ollama + OpenAI + Python

## 1. 指定模型名称

如果你使用的模型不是 "phi3:mini"，请在下面的单元格中更改该变量的值。这个变量将在整个笔记本的代码中使用。


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. 设置 Open AI 客户端

通常，OpenAI 客户端用于与 OpenAI.com 或 Azure OpenAI 交互，以使用大型语言模型。  
然而，它也可以与 Ollama 一起使用，因为 Ollama 提供了一个与 OpenAI 兼容的端点，地址为 "http://localhost:11434/v1"。


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. 生成聊天回复

现在我们可以使用 OpenAI SDK 为对话生成回复。这个请求应该生成一首关于猫的俳句：


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. 提示工程

发送给语言模型的第一条消息被称为“系统消息”或“系统提示”，它为模型设定了整体指令。
你可以提供自定义的系统提示，以引导语言模型以不同的方式生成输出。
修改下面的 `SYSTEM_MESSAGE`，让模型像你最喜欢的电影/电视角色一样回答，或者从 [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts) 中获取其他系统提示的灵感。

一旦你自定义了系统消息，请在 `USER_MESSAGE` 中提供第一个用户问题。


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. 少量示例

另一种引导语言模型的方法是提供“少量示例”，即一系列问题/答案示例，展示模型应该如何回应。

下面的示例尝试让语言模型扮演教学助理的角色，通过提供一些教学助理可能会给出的问答示例，然后用学生可能会问的问题来提示模型。

先试试这个方法，然后修改 `SYSTEM_MESSAGE`、`EXAMPLES` 和 `USER_MESSAGE` 来适应新的场景。


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. 检索增强生成

RAG（检索增强生成）是一种技术，通过先从知识源中检索相关信息，然后基于这些信息生成回答，使语言模型能够准确回答特定领域的问题。

我们提供了一个包含混合动力汽车数据的本地 CSV 文件。以下代码会读取该 CSV 文件，搜索与用户问题匹配的信息，然后基于找到的信息生成回答。请注意，这个过程会比之前的示例耗时更长，因为它向模型发送了更多数据。如果你发现答案仍然没有基于数据，可以尝试进行系统工程或使用其他模型。通常，RAG 在使用更大的模型或经过微调的 SLM 时效果更佳。


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**免责声明**：  
本文档使用AI翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于重要信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
